# 02 报文解析

In [1]:
import numpy as np
import pandas as pd
import binascii
import datetime
import math

In [2]:
import Ipynb_importer
from public_function import *

importing Jupyter notebook from public_function.ipynb


# 公共函数

```
# 8进制转10进制
def oct2dec(x):
    output = int(str(x), 8)
    return output

# 16进制转10进制（先乘精度，后偏移）
def hex2dec(x, n=0, k=1, e=False):
    x = x.upper()
    k_n = 0 if k>=1 else int(math.log10(k))
    if e:
        if x == "EF" or x == "FFFE" or x == "FFFFFFFE":
            return "异常"
        elif x == "FF" or x == "FFFF" or x == "FFFFFFFF":
            return "无效"
        else:
            output_s = int(str(x), 16)
            output = output_s*k + n
            return round(output, -k_n)
    else :
        output_s = int(str(x), 16)
        output = output_s*k + n
        return round(output, -k_n)
    
    

# 16进制转字符串
def hex2str(x):
    output = binascii.a2b_hex(x).decode("utf8")
    return output

# 字符串分割
def str_split(data, n):
    global data_s, data_f, data_2
    data_s = data[:n]
    data_f = data[n:]
    data_2 = data_f[0:2]

# 时间
def bw_datetime(x):
    year = hex2dec(x[0:2]) + 2000
    month = hex2dec(x[2:4])
    day = hex2dec(x[4:6])
    hour = hex2dec(x[6:8])
    minute = hex2dec(x[8:10])
    second = hex2dec(x[10:12])
    output = datetime.datetime(year, month, day, hour, minute, second).strftime('%Y-%m-%d %H:%M:%S')
    return output
```

```
# 字符串类 直接替换
# 列表类替换

def dict_list_replace(n, x):
    x = x.upper()
    index = jx_dict_o[n].index(x)
    output = jx_dict[n][index]
    return output

# 多值划分为列表
def hex2list(x, n=1, hn=0, hk=1, he=False):
    x = x.upper()
    l = int(len(x)/(2*n))
    output = []
    for i in range(0,l):
        s_16 = x[2*n*i:2*n*i + 2*n]
        s_10 = hex2dec(s_16, n=hn, k=hk, e=he)
        output.append(s_10)
    return output
```

# 解析字典

In [3]:
jx_dict_o = {
    "00_02":["01", "02", "03", "04", "05", "06"],
    "00_03":["01", "02", "03", "FE"],
    "00_05":["01", "02", "03", "FE", "FF"],
    "00_07_02_01_01":["01", "02", "03", "FE", "FF"],
    "00_07_02_01_02":["01", "02", "03", "04", "FE", "FF"],
    "00_07_02_01_03":["01", "02", "03", "FE", "FF"],
    "00_07_02_01_06":["01", "02", "FE", "FF"],
    "00_07_02_02_01":["01", "02", "03", "04", "FE", "FF"],
    "00_07_02_03_12":["01", "02", "FE", "FF"],
    "00_07_02_04_01":["01", "02", "FE", "FF"],
    "00_07_05_05":["01", "02", "03", "FE", "FF"],
}

jx_dict = {
    "00_02":["车辆登入", "实时信息上报", "补发信息上报", "车辆登出", "平台登入", "平台登出"],
    "00_03":["成功", "错误", "VIN重复", "命令"],
    "00_05":["不加密", "RSA加密", "AES128位加密", "异常", "无效"],
    "00_07_02_01_01":["启动", "熄火", "其他", "异常", "无效"],
    "00_07_02_01_02":["停车充电", "行驶充电", "未充电", "充电完成", "异常", "无效"],
    "00_07_02_01_03":["纯电", "混动", "燃油", "异常", "无效"],
    "00_07_02_01_06":["工作", "断开", "异常", "无效"],
    "00_07_02_02_01":["耗电", "发电", "关闭", "准备", "异常", "无效"],
    "00_07_02_03_12":["工作", "断开", "异常", "无效"],
    "00_07_02_04_01":["启动", "关闭", "异常", "无效"],
    "00_07_05_05":["不加密", "RSA加密", "AES128位加密", "异常", "无效"],
}

# 全局变量

In [4]:
data_dict_jx = {}

# 解析函数

In [5]:
# 00 基础数据
def fun_jx_00(data):
    global data_dict_jx
    data_dict_jx["00_01 起始符"] = hex2str(data["起始符"])
    data_dict_jx["00_02 命令标识"] = dict_list_replace('00_02', data['命令标识'])
    data_dict_jx["00_03 应答标志"] = dict_list_replace('00_03', data['应答标志'])
    data_dict_jx["00_04 唯一识别码"] = hex2str(data["唯一识别码"])
    data_dict_jx["00_05 数据单元加密方式"] = dict_list_replace('00_05', data['数据单元加密方式'])
    data_dict_jx["00_06 数据单元长度"] = hex2dec(data["数据单元长度"])
    data_dict_jx["00_08 效验码"] = data["效验码"]

In [6]:
# 00_07 数据单元

def fun_jx_00_07(data):
    global data_dict_jx
    data_dict_jx['00_07 数据单元'] = {}
    # 命令标识
    mark = data["命令标识"]
    
    if mark == '01':
        fun_jx_00_07_01(data)
    elif mark == '02' or mark == '03':
        fun_jx_00_07_02(data)
    elif mark == '04':
        fun_jx_00_07_04(data)
    elif mark == '05':
        fun_jx_00_07_05(data)
    elif mark == '06':
        fun_jx_00_07_06(data)
    else :
        print("ERROR:命令标识错误！")

In [7]:
# 01 车辆登入
def fun_jx_00_07_01(data):
    global data_dict_jx
    
    data_o = data['数据单元']['车辆登入']
    data_jx = {}
    
    data_jx['01 数据采集时间'] = bw_datetime(data_o['数据采集时间'])
    data_jx['02 登入流水号'] = hex2dec(data_o['登入流水号'])
    data_jx['03 ICCID'] = hex2str(data_o['ICCID'])
    data_jx['04 可充电储能子系统数'] = hex2dec(data_o['可充电储能子系统数'])
    data_jx['05 可充电储能系统编码长度'] = hex2dec(data_o['可充电储能系统编码长度'])
    data_jx['06 可充电储能系统编码'] = fun_jx_00_07_01_06(data_o)

    data_dict_jx['00_07 数据单元']['01 车辆登入'] = {}
    data_dict_jx['00_07 数据单元']['01 车辆登入'] = data_jx

    
def fun_jx_00_07_01_06(data):
    
    data_no = data['可充电储能系统编码']
    
    if data_no == "NA" or data_no == "" or data_no == ["NA"] or data_no == [""] :
        return "NA"
    else :
        dianchi_n = hex2dec(data['可充电储能子系统数'])
        dianchi_len = hex2dec(data['可充电储能系统编码长度'])
        
        data_no_jx = []
        
        for i in range(0,dianchi_n):
            dianchi_i = hex2str(data_no[i*2*dianchi_len: i*2*dianchi_len + 2*dianchi_len])
            data_no_jx.append(dianchi_i)
        return data_no_jx


In [8]:
# 02 实时信息上报
def fun_jx_00_07_02(data):
    global data_dict_jx
    
    data_o = data["数据单元"]["信息上报"]
    
    data_dict_jx['00_07 数据单元']['02 信息上报'] = {}
    data_dict_jx['00_07 数据单元']['02 信息上报']["00 数据采集时间"] = bw_datetime(data_o["数据采集时间"])
    
    data["数据单元"]["信息上报"]
    
    for i in data_o:
        if i == "整车数据":
            fun_jx_00_07_02_01(data)
        elif i == "驱动电机数据":
            fun_jx_00_07_02_02(data)
        elif i == "燃料电池数据":
            fun_jx_00_07_02_03(data)
        elif i == "发动机数据":
            fun_jx_00_07_02_04(data)
        elif i == "车辆位置数据":
            fun_jx_00_07_02_05(data)
        elif i == "极值数据":
            fun_jx_00_07_02_06(data)
        elif i == "报警数据":
            fun_jx_00_07_02_07(data)
        elif i == "可充电储能装置电压数据":
            fun_jx_00_07_02_08(data)
        elif i == "可充电储能装置温度数据":
            fun_jx_00_07_02_09(data)
    

In [9]:
# 02_01 整车数据
def fun_jx_00_07_02_01(data):
    global data_dict_jx
    
    data_o = data['数据单元']['信息上报']['整车数据']
    data_jx = {}
    
    data_jx['01 车辆状态'] = dict_list_replace("00_07_02_01_01", data_o['车辆状态'])
    data_jx['02 充电状态'] = dict_list_replace("00_07_02_01_02", data_o['充电状态'])
    data_jx['03 运行模式'] = dict_list_replace("00_07_02_01_03", data_o['运行模式'])
    data_jx['04 车速'] = hex2dec(data_o['车速'], k=0.1)
    data_jx['05 累计里程'] = hex2dec(data_o['累计里程'], k=0.1)
    data_jx['06 总电压'] = hex2dec(data_o['总电压'], k=0.1)
    data_jx['07 总电流'] = hex2dec(data_o['总电流'], n=-1000, k=0.1)
    data_jx['08 SOC'] = hex2dec(data_o['SOC'])
    data_jx['09 DC-DC状态'] = dict_list_replace("00_07_02_01_06", data_o['DC-DC状态'])
    data_jx['10 挡位'] = fun_jx_00_07_02_01_10(data_o['挡位'])
    data_jx['11 绝缘电阻'] = hex2dec(data_o['绝缘电阻'])
    data_jx['12 加速踏板行程值'] = fun_jx_00_07_02_01_12(data_o['加速踏板行程值'])
    data_jx['13 制动踏板状态'] = fun_jx_00_07_02_01_13(data_o['制动踏板状态'])
    
    data_dict_jx['00_07 数据单元']['02 信息上报']['01 整车数据'] = {}
    data_dict_jx['00_07 数据单元']['02 信息上报']['01 整车数据'] = data_jx
    
# 02_01_10 挡位
def fun_jx_00_07_02_01_10(data):
    n = '{:08b}'.format(int(data, 16))    
    qudongli = n[-6]
    zhidongli = n[-5]
    dangwei = n[-4:]
    
    # 挡位
    if dangwei == '0000':
        dangwei_s = '空挡'
    elif dangwei == '1101':
        dangwei_s = '倒挡'
    elif dangwei == '1110':
        dangwei_s = '自动D挡'
    elif dangwei == '1111':
        dangwei_s = '停车P挡'
    else :
        dangwei_s = (str(int(dangwei, 2)) + "档" )
        
    # 驱动力
    if qudongli == "1":
        qudongli_s = "有驱动力"
    else :
        qudongli_s = "无驱动力"
        
    # 制动力
    if zhidongli == "1":
        zhidongli_s = "有制动力"
    else :
        zhidongli_s = "无制动力"    
    
    output = [n, dangwei_s, zhidongli_s, qudongli_s]
    return output

# 02_01_12 加速踏板行程值
def fun_jx_00_07_02_01_12(data):
    if data == 'FE':
        return "异常"
    elif data == "FF":
        return "无效"
    else :
        return hex2dec(data)

# 02_01_13 制动踏板状态
def fun_jx_00_07_02_01_13(data):
    if data == 'FE':
        return "异常"
    elif data == "FF":
        return "无效"
    elif data == "65":
        return "制动有效"
    else :
        return hex2dec(data)
    

In [10]:
# 02_02 驱动电机数据
def fun_jx_00_07_02_02(data):
    global data_dict_jx
    data_o = data['数据单元']['信息上报']['驱动电机数据']
    data_o_o = data['数据单元']['信息上报']['驱动电机数据']['驱动电机总成信息列表']
    
    data_jx = {}
    data_jx['01 驱动电机个数'] = hex2dec(data_o['驱动电机个数'])
    data_jx['02 驱动电机总成信息列表'] = {}
    
    for i in data_o['驱动电机总成信息列表']:
        data_jx['02 驱动电机总成信息列表'][i] = {
            '01 驱动电机状态' : dict_list_replace("00_07_02_02_01", data_o_o[i]['驱动电机状态']),
            '02 驱动电机控制器温度' : hex2dec(data_o_o[i]['驱动电机控制器温度'], n=-40),
            '03 驱动电机转速' : hex2dec(data_o_o[i]['驱动电机转速'], n=-20000),
            '04 驱动电机转矩' : hex2dec(data_o_o[i]['驱动电机转矩'], n=-2000, k=0.1),
            '05 驱动电机温度' : hex2dec(data_o_o[i]['驱动电机温度'], n=-40),
            '06 电机控制器输入电压' : hex2dec(data_o_o[i]['电机控制器输入电压'], k=0.1),
            '07 电机控制器直流母线电流' : hex2dec(data_o_o[i]['电机控制器直流母线电流'], n=-1000, k=0.1)
        }
    
    data_dict_jx['00_07 数据单元']['02 信息上报']['02 驱动电机数据'] = {}
    data_dict_jx['00_07 数据单元']['02 信息上报']['02 驱动电机数据'] = data_jx


In [11]:
# 02_03 

def fun_jx_00_07_02_03(data):
    global data_dict_jx
    data_o = data['数据单元']['信息上报']['燃料电池数据']
    
    data_jx = {}
    
    data_jx['01 燃料电池电压'] = hex2dec(data_o['燃料电池电压'], k=0.1, e=True)
    data_jx['02 燃料电池电流'] = hex2dec(data_o['燃料电池电流'], k=0.1, e=True)
    data_jx['03 燃料消耗率'] = hex2dec(data_o['燃料消耗率'], k=0.01, e=True)
    data_jx['04 燃料电池温度探针总数N'] = hex2dec(data_o['燃料电池温度探针总数N'], e=True)
    data_jx['05 探针温度值'] = hex2list(data_o['探针温度值'], n=1, hn=-40)
    data_jx['06 氢系统中最高温度'] = hex2dec(data_o['总电压'], k=0.1)
    data_jx['07 氢系统中最高温度探针代号'] = hex2dec(data_o['氢系统中最高温度探针代号'], e=True)
    data_jx['08 氢气最高浓度'] = hex2dec(data_o['氢气最高浓度'], e=True)
    data_jx['09 氢气最高浓度传感器代号'] = hex2dec(data_o['氢气最高浓度传感器代号'], e=True)
    data_jx['10 氢气最高压力'] = hex2dec(data_o['氢气最高压力'], k=0.1)
    data_jx['11 氢气最高压力传感器代号'] = hex2dec(data_o['氢气最高压力传感器代号'], e=True)
    data_jx['12 高压DC/DC状态'] = dict_list_replace("00_07_02_03_12",data_o['高压DC/DC状态'])
    
    data_dict_jx['00_07 数据单元']['02 信息上报']['03 燃料电池数据'] = {}
    data_dict_jx['00_07 数据单元']['02 信息上报']['03 燃料电池数据'] = data_jx
    

In [12]:
# 02_04 发动机数据
def fun_jx_00_07_02_04(data):
    global data_dict_jx
    data_o = data['数据单元']['信息上报']['发动机数据']
    
    data_jx = {}
    
    data_jx['01 发动机状态'] = dict_list_replace("00_07_02_04_01", data_o['发动机状态'])
    data_jx['02 曲轴转速'] = hex2dec(data_o['曲轴转速'], e=True)
    data_jx['03 燃料消耗率'] = hex2dec(data_o['燃料消耗率'], k=0.01, e=True)
    
    data_dict_jx['00_07 数据单元']['02 信息上报']['04 发动机数据'] = {}
    data_dict_jx['00_07 数据单元']['02 信息上报']['04 发动机数据'] = data_jx
    

In [13]:
# 02_05 车辆位置数据

def fun_jx_00_07_02_05(data):
    global data_dict_jx
    data_o = data['数据单元']['信息上报']['车辆位置数据']
    data_jx = {}
    
    data_jx['01 定位状态'] = fun_jx_00_07_02_05_01(data_o['定位状态'])
    data_jx['02 经度'] = hex2dec(data_o['经度'], k=0.000001)
    data_jx['03 纬度'] = hex2dec(data_o['纬度'], k=0.000001)
    
    data_dict_jx['00_07 数据单元']['02 信息上报']['05 车辆位置数据'] = {}
    data_dict_jx['00_07 数据单元']['02 信息上报']['05 车辆位置数据'] = data_jx
    
def fun_jx_00_07_02_05_01(data):
    n = '{:08b}'.format(int(data, 16))    
    state = n[-1]
    longitude = n[-2]
    latitude = n[-3]
    
    if state == '0':
        state_s = "定位有效"
    else :
        state_s = "定位无效"
    
    if longitude == '0':
        longitude_s = "北纬"
    else :
        longitude_s = "南纬"
        
    if latitude == '0':
        latitude_s = "东经"
    else :
        latitude_s = "西经"
    
    output = [n, state_s, longitude_s, latitude_s]
    return output

In [14]:
# 02_06 极值数据

def fun_jx_00_07_02_06(data):
    global data_dict_jx
    data_o = data['数据单元']['信息上报']['极值数据']
    data_jx = {}
    
    data_jx['01 最高电压电池子系统号'] = hex2dec(data_o['最高电压电池子系统号'], e=True)
    data_jx['02 最高电压电池单体代号'] = hex2dec(data_o['最高电压电池单体代号'], e=True)
    data_jx['03 电池单体电压最高值'] = hex2dec(data_o['电池单体电压最高值'], k=0.001, e=True)
    data_jx['04 最低电压电池子系统号'] = hex2dec(data_o['最低电压电池子系统号'], e=True)
    data_jx['05 最低电压电池单体代号'] = hex2dec(data_o['最低电压电池单体代号'], e=True)
    data_jx['06 电池单体电压最低值'] = hex2dec(data_o['电池单体电压最低值'], k=0.001, e=True)
    data_jx['07 最高温度子系统号'] = hex2dec(data_o['最高温度子系统号'], e=True)
    data_jx['08 最高温度探针序号'] = hex2dec(data_o['最高温度探针序号'], e=True)
    data_jx['09 最高温度值'] = hex2dec(data_o['最高温度值'], n=-40, e=True)
    data_jx['10 最低温度子系统号'] = hex2dec(data_o['最低温度子系统号'], e=True)
    data_jx['11 最低温度探针序号'] = hex2dec(data_o['最低温度探针序号'], e=True)
    data_jx['12 最低温度值'] = hex2dec(data_o['最低温度值'], n=-40, e=True)
    
    data_dict_jx['00_07 数据单元']['02 信息上报']['06 极值数据'] = {}
    data_dict_jx['00_07 数据单元']['02 信息上报']['06 极值数据'] = data_jx


In [15]:
# 02_07 报警数据（待优化）

def fun_jx_00_07_02_07(data):
    global data_dict_jx
    data_o = data['数据单元']['信息上报']['报警数据']
    data_jx = {}
    
    data_jx['01 最高报警等级'] = hex2dec(data_o['最高报警等级'], e=True)
    data_jx['02 通用报警标志'] = fun_jx_00_07_02_07_02(data_o['通用报警标志'])
    data_jx['03 可充电储能装置故障总数N1'] = hex2dec(data_o['可充电储能装置故障总数N1'], e=True)
    data_jx['04 驱动电机故障总数N2'] = hex2dec(data_o['驱动电机故障总数N2'], e=True)
    data_jx['05 发动机故障总数N3'] = hex2dec(data_o['发动机故障总数N3'], e=True)
    data_jx['06 其他故障总数N4'] = hex2dec(data_o['其他故障总数N4'], e=True)
    
    data_dict_jx['00_07 数据单元']['02 信息上报']['07 报警数据'] = {}
    data_dict_jx['00_07 数据单元']['02 信息上报']['07 报警数据'] = data_jx

def fun_jx_00_07_02_07_02(data):
    n = '{:032b}'.format(int(data, 16))
    
    baojing_list = [
        "温度差异报警",
        "电池高温报警",
        "车载储能装置类型过压报警",
        "车载储能装置类型欠压报警",
        "SOC低报警",
        "单体电池过压报警",
        "单体电池欠压报警",
        "SOC过高报警",
        "SOC跳变报警",
        "可充电储能系统不匹配报警",
        "电池单体一致性差报警",
        "绝缘报警",
        "DC-DC温度报警",
        "制动系统报警",
        "DC-DC状态报警",
        "驱动电机控制器温度报警",
        "高压互锁状态报警",
        "驱动电机温度报警",
        "车载储能装置类型过充",
    ]
    
    baojing = [n]
    
    for i in range(0,19):
        if n[-i] == "1":
            baojing.append(baojing_list[i])
    
    return baojing


In [16]:
# 02_08 可充电储能装置电压数据

def fun_jx_00_07_02_08(data):
    global data_dict_jx
    data_o = data['数据单元']['信息上报']['可充电储能装置电压数据']
    data_o_o = data['数据单元']['信息上报']['可充电储能装置电压数据']['可充电储能子系统电压信息列表']
    data_jx = {}
    
    data_jx = {}
    data_jx['01 可充电储能子系统个数'] = hex2dec(data_o['可充电储能子系统个数'], e=True)
    data_jx['02 可充电储能子系统电压信息列表'] = {}
    
    for i in data_o['可充电储能子系统电压信息列表']:
        data_jx['02 可充电储能子系统电压信息列表'][i] = {
            '01 可充电储能装置电压' : hex2dec(data_o_o[i]['可充电储能装置电压']),
            '02 可充电储能装置电流' : hex2dec(data_o_o[i]['可充电储能装置电流'], k=0.1, e=True),
            '03 单体电池总数' : hex2dec(data_o_o[i]['单体电池总数'], e=True),
            '04 本帧起始电池序号' : hex2dec(data_o_o[i]['本帧起始电池序号']),
            '05 本帧单体电池总数' : hex2dec(data_o_o[i]['本帧单体电池总数']),
            '06 单体电池电压' : hex2list(data_o_o[i]['单体电池电压'], n=2, hk=0.001, he=True)
        }
    
    data_dict_jx['00_07 数据单元']['02 信息上报']['08 可充电储能装置电压数据'] = {}
    data_dict_jx['00_07 数据单元']['02 信息上报']['08 可充电储能装置电压数据'] = data_jx


In [17]:
# 02_09

def fun_jx_00_07_02_09(data):
    global data_dict_jx
    data_o = data['数据单元']['信息上报']['可充电储能装置温度数据']
    data_o_o = data['数据单元']['信息上报']['可充电储能装置温度数据']['可充电储能子系统温度信息列表']
    data_jx = {}
    
    data_jx = {}
    data_jx['01 可充电储能子系统个数'] = hex2dec(data_o['可充电储能子系统个数'], e=True)
    data_jx['02 可充电储能子系统温度信息列表'] = {}
    
    for i in data_o['可充电储能子系统温度信息列表']:
        data_jx['02 可充电储能子系统温度信息列表'][i] = {
            '01 可充电储能温度探针个数' : hex2dec(data_o_o[i]['可充电储能温度探针个数']),
            '02 可充电储能子系统各温度探针检测到的温度值' : hex2list(data_o_o[i]['可充电储能子系统各温度探针检测到的温度值'], n=1, hn=-40, he=True)
        }
    
    data_dict_jx['00_07 数据单元']['02 信息上报']['09 可充电储能装置温度数据'] = {}
    data_dict_jx['00_07 数据单元']['02 信息上报']['09 可充电储能装置温度数据'] = data_jx


In [18]:
# 03 补发信息上报（同 02实时）

In [19]:
# 04 车辆登出

def fun_jx_00_07_04(data):
    global data_dict_jx
    
    data_o = data['数据单元']['车辆登出']
    data_jx = {}
    
    data_jx['01 登出时间'] = bw_datetime(data_o['登出时间'])
    data_jx['02 登出流水号'] = hex2dec(data_o['登出流水号'])
    
    data_dict_jx['00_07 数据单元']['04 车辆登出'] = {}
    data_dict_jx['00_07 数据单元']['04 车辆登出'] = data_jx
    

In [20]:
# 05 平台登入

def fun_jx_00_07_05(data):
    global data_dict_jx
    
    data_o = data['数据单元']['平台登入']
    data_jx = {}
    
    data_jx['01 登出时间'] = bw_datetime(data_o['登出时间'])
    data_jx['02 登出流水号'] = hex2dec(data_o['登出流水号'])
    data_jx['03 平台用户名'] = hex2str(data_o['平台用户名'])
    data_jx['04 平台密码'] = hex2str(data_o['平台密码'])
    data_jx['05 加密规则'] = dict_list_replace("00_07_05_05", data_o['加密规则'])
    
    data_dict_jx['00_07 数据单元']['06 平台登出'] = {}
    data_dict_jx['00_07 数据单元']['06 平台登出'] = data_jx


In [21]:
# 06 平台登出

def fun_jx_00_07_06(data):
    global data_dict_jx
    
    data_o = data['数据单元']['平台登出']
    data_jx = {}
    
    data_jx['01 登出时间'] = bw_datetime(data_o['登出时间'])
    data_jx['02 登出流水号'] = hex2dec(data_o['登出流水号'])
    
    data_dict_jx['00_07 数据单元']['06 平台登出'] = {}
    data_dict_jx['00_07 数据单元']['06 平台登出'] = data_jx
    

# 执行解析

In [22]:
def fun_jx(data):
    global data_dict_jx
    fun_jx_00(data)
    
    fun_jx_00_07(data)
    
    return data_dict_jx